# 구글 드라이브 연동하기
모델 체크포인트 등을 저장해 둘 구글 드라이브를 연결합니다. 자신의 구글 계정에 적용됩니다.

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
!pip install transformers==4.28.1 pytorch-lightning==1.6.1

# 모델 로딩
파인튜닝을 마친 모델과 토크나이저를 읽어 들입니다.

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# 저장된 모델과 토크나이저 불러오기
model_path = "/gdrive/MyDrive/kdt_231026/m7_nlp응용/data/paircl_bert_model2"  # 모델을 저장한 경로로 변경해야 함
model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(model_path)

# 인퍼런스 함수 선언
인퍼런스 함수를 선언합니다.

In [ ]:
def inference_fn(premise, hypothesis):
    inputs = tokenizer(
        premise,
        hypothesis,
        return_tensors="pt",  # PyTorch 텐서로 반환하도록 설정
        max_length=128,  # 직접 지정한 최대 시퀀스 길이
        padding="max_length",
        truncation=True,
    )

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        prob = torch.softmax(logits, dim=1)
        entailment_prob = round(prob[0][1].item(), 2) # 해당 샘플을 "참(Entailment)"으로 분류할 확률
        contradiction_prob = round(prob[0][0].item(), 2) # "거짓(Contradiction)"으로 분류할 확률

        pred_idx = torch.argmax(prob, dim=1).item()
        if pred_idx == 1:
            pred = "참 (entailment)"
        else:
            pred = "거짓 (contradiction)"

    return {
        "premise": premise,
        "hypothesis": hypothesis,
        "prediction": pred,
        "entailment_data": f"참 {entailment_prob}",
        "contradiction_data": f"거짓 {contradiction_prob}",
        "entailment_width": f"{entailment_prob * 100}%",
        "contradiction_width": f"{contradiction_prob * 100}%",
    }
premise = 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .'
hypothesis = 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .'

print(inference_fn(premise, hypothesis))

{'premise': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .', 'hypothesis': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .', 'prediction': '참 (entailment)', 'entailment_data': '참 0.94', 'contradiction_data': '거짓 0.06', 'entailment_width': '94.0%', 'contradiction_width': '6.0%'}


In [ ]:
premise = 'The sky is full of clouds.'
hypothesis = "It looks like it's going to be sunny."
print(inference_fn(premise, hypothesis))

{'premise': 'The sky is full of clouds.', 'hypothesis': "It looks like it's going to be sunny.", 'prediction': '거짓 (contradiction)', 'entailment_data': '참 0.4', 'contradiction_data': '거짓 0.6', 'entailment_width': '40.0%', 'contradiction_width': '60.0%'}
